Part 1: Analyzing Best-Seller Trends Across Product Categories
Crosstab Analysis:

Create a crosstab between the product category and the isBestSeller status.

Are there categories where being a best-seller is more prevalent?

Hint: one option is to calculate the proportion of best-sellers for each category and then sort the categories based on this proportion in descending order.



In [9]:
import pandas as pd
import plotly as plt
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import numpy as np


ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [2]:
df = pd.read_csv('amz_uk_price_prediction_dataset.csv')

In [3]:
df.head(10)

,uid,asin,title,stars,reviews,price,isBestSeller,boughtInLastMonth,category
0,1,B09B96TG33,"Echo Dot (5th generation, 2022 release) | Big ...",4.7,15308,21.99,False,0,Hi-Fi Speakers
1,2,B01HTH3C8S,"Anker Soundcore mini, Super-Portable Bluetooth...",4.7,98099,23.99,True,0,Hi-Fi Speakers
2,3,B09B8YWXDF,"Echo Dot (5th generation, 2022 release) | Big ...",4.7,15308,21.99,False,0,Hi-Fi Speakers
3,4,B09B8T5VGV,"Echo Dot with clock (5th generation, 2022 rele...",4.7,7205,31.99,False,0,Hi-Fi Speakers
4,5,B09WX6QD65,Introducing Echo Pop | Full sound compact Wi-F...,4.6,1881,17.99,False,0,Hi-Fi Speakers
5,6,B09B97WSLF,"Echo Dot with clock (5th generation, 2022 rele...",4.7,7205,31.99,False,0,Hi-Fi Speakers
6,7,B09B9615X2,"Echo Dot (5th generation, 2022 release) | Big ...",4.7,15308,21.99,False,0,Hi-Fi Speakers
7,8,B01MTB55WH,Anker Soundcore 2 Portable Bluetooth Speaker w...,4.7,103673,39.99,False,0,Hi-Fi Speakers
8,9,B016MO90GW,"Bluetooth Speaker, Anker Soundcore Speaker Upg...",4.7,29909,29.99,False,0,Hi-Fi Speakers
9,10,B09ZQF5GZW,RIENOK Bluetooth Speaker Portable Wireless Blu...,4.5,16014,29.99,False,0,Hi-Fi Speakers


In [4]:
# Create a crosstab between the product category and the isBestSeller status.
pd.crosstab(df['category'], df['isBestSeller'])

isBestSeller,False,True
category,,
3D Printers,247,1
3D Printing & Scanning,4065,2
Abrasive & Finishing Products,245,5
Action Cameras,1696,1
Adapters,251,3
...,...,...
Wind Instruments,243,7
Window Treatments,234,5
Women,17559,213


In [5]:
# Are there categories where being a best-seller is more prevalent?
pd.crosstab(df['category'], df['isBestSeller'], normalize='index')

isBestSeller,False,True
category,,
3D Printers,0.995968,0.004032
3D Printing & Scanning,0.999508,0.000492
Abrasive & Finishing Products,0.980000,0.020000
Action Cameras,0.999411,0.000589
Adapters,0.988189,0.011811
...,...,...
Wind Instruments,0.972000,0.028000
Window Treatments,0.979079,0.020921
Women,0.988015,0.011985


2.  Statistical Tests:

Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.
Compute Cramér's V to understand the strength of association between best-seller status and category.


In [6]:
#Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.
from scipy.stats import chi2_contingency
chi2, p, dof, ex = chi2_contingency(pd.crosstab(df['category'], df['isBestSeller']))

chi2, p, dof


(36540.20270061387, 0.0, 295)

In [7]:
#Compute Cramér's V to understand the strength of association between best-seller status and category.
import numpy as np
n = df.shape[0]
v = np.sqrt(chi2 / (n * (min(pd.crosstab(df['category'], df['isBestSeller']).shape) - 1)))
v   # 0.02

0.1222829439760564

In [8]:
#Visualize the relationship between product categories and the best-seller status using a stacked bar chart.
pd.crosstab(df['category'], df['isBestSeller']).plot(kind='bar', stacked=True)


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

Part 2: Exploring Product Prices and Ratings Across Categories and Brands

Objective: Investigate how different product categories influence product prices.

In [ ]:
#Preliminary Step: Remove outliers in product prices. 
df['price'].describe()
df['price'].quantile(0.99)

1408.82

In [ ]:
#Violin Plots:
#Use a violin plot to visualize the distribution of price across different product categories. Filter out the top 20 categories based on count for better visualization.
top_categories = df['category'].value_counts().head(20).index
df_top_categories = df[df['category'].isin(top_categories)]
plt.express.violin(df_top_categories, x='category', y='price')


AttributeError: module 'plotly' has no attribute 'express'

In [ ]:
#Which product category tends to have the highest median price? Don't filter here by top categories.
df.groupby('category')['price'].median().sort_values(ascending=False).head(1)


category
Laptops    1042.725
Name: price, dtype: float64

In [ ]:
#Bar Charts:
# Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
top_categories = df['category'].value_counts().head(10).index
df_top_categories = df[df['category'].isin(top_categories)]
df_top_categories.groupby('category')['price'].mean().sort_values().plot(kind='bar')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [10]:
# Which product category commands the highest average price? Don't filter here by top categories.
df.groupby('category')['price'].mean().sort_values().plot(kind='bar')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

Objective: Analyze how product ratings (stars) correlate with product prices.

Correlation Coefficients:

Calculate the correlation coefficient between price and stars.
Is there a significant correlation between product price and its rating?

In [13]:
df_no_outliers = df[df['price'] <= df['price'].quantile(0.99)]

In [16]:
#Calculate the correlation coefficient between price and stars.
correlation_price_rating = df_no_outliers[['price', 'stars']].corr()
correlation_price_rating


,price,stars
price,1.000000,-0.143055
stars,-0.143055,1.000000


In [17]:
#Is there a significant correlation between product price and its rating?
from scipy.stats import pearsonr
pearsonr(df['price'], df['stars'])


PearsonRResult(statistic=-0.12490673262148647, pvalue=0.0)

Visualizations:

Use a scatter plot to visualize the relationship between product rating and price. 

What patterns can you observe?

Use a correlation heatmap to visualize correlations between all numerical variables.
Examine if product prices typically follow a normal distribution using a QQ plot.
